In [0]:
!pip install shap

In [0]:
import pprint
import yaml
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import numpy as np
import pandas as pd

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from modelling.modelling import (
    build_linear_baseline,
    build_logistic_regression_model,
    build_random_forest_model,
    run_all_models,
)

from modelling.modelling_utils import (
    remove_collinear_features,
    run_shap_analysis,
    create_stakeholder_report,
    print_model_summary,
    get_significant_features,
    export_results,
    plot_shap_summary,
    plot_shap_bar,
    plot_shap_dependence,
)

# Load configs
with open('../configs/statistical_tests.yaml', 'r') as f:
    stats_config = yaml.safe_load(f)

with open('../configs/data_paths.yaml', 'r') as f:
    paths_config = yaml.safe_load(f)

pd.set_option('display.max_columns', None)
print("✓ Imports and configs loaded successfully")

In [0]:
# Loading user level dataframe
notebook_path = os.getcwd()
repo_root = os.path.abspath(os.path.join(notebook_path, ".."))
misc_dir = os.path.join(repo_root, "misc")

user_df_input_path = os.path.join(misc_dir,
                           os.path.basename(paths_config['output_files']['user_info_df_post_eda']))

test_results_df_input_path = os.path.join(misc_dir,
                           os.path.basename(paths_config['output_files']['test_results_df']))

user_info_df = pd.read_parquet(user_df_input_path)
test_results_df = pd.read_parquet(test_results_df_input_path)

user_info_df = user_info_df[~user_info_df['wonky_study_count'].isna()]
print(f"✓ Data loaded: {user_info_df.shape}")

In [0]:
user_info_df = user_info_df[~user_info_df['wonky_study_count'].isna()]
user_info_df.shape

#### Stage 1 - VIF & Correlation checks

In [0]:
test_results_df

In [0]:
# significant_features = test_results_df[test_results_df['significant_both']]['feature'].tolist()
# significant_features = [item for item in significant_features if 'is_weekend' not in item]
# significant_features = significant_features + ['risk', 'quality']

# print(f"Significant features from testing: {len(significant_features)}")
# pprint.pprint(significant_features)

In [0]:
significant_features = test_results_df['feature'].tolist()
significant_features = [item for item in significant_features if 'is_weekend' not in item]

print(f"Significant features from testing: {len(significant_features)}")
pprint.pprint(significant_features)

In [0]:
significcant_features_to_remove = [
  # 'share_location_data_False',
  # 'email_verified_False',
  # 'is_6am',
  # 'is_business_hour',
  # 'is_business_hour_weekend',
  # 'is_business_hour_weekend',
  # 'is_weekend',
  # 'is_early_morning',
  # 'is_morning',
  # 'is_night', 
  # 'is_afternoon',
  # 'is_evening',
  # 'risk_round10_100',
  # 'risk_round10_70',
  # 'risk_round10_0',
  # 'quality_round10_100',
  # 'quality_round10_70',
  # 'quality_round10_0',
  # 'days_active_11_to_20',
  # 'days_active_76_to_100',
  # 'days_active_151_to_200',
  # 'days_active_101_to_125',
  # 'days_active_51_to_75',
  # 'days_active_21_to_30',
  # 'days_active_0_to_2',
  # 'days_active_3_to_10',
  # 'days_active_126_to_150',
  # 'days_active_before_task',
  # 'days_active_201_to_250',
  # 'days_active_31_to_50',
  # 'days_active_251_plus',
  # 'total_tasks_126_175',
  # 'total_tasks_26_75',
  # 'total_tasks_11_25',
  # 'total_tasks_176_225',
  # 'total_tasks_226_plus',
  # 'total_tasks_1_10',
  # 'total_tasks_76_125',
  # 'ditr_os_ios',
  # 'is_monday',
  # 'hardware_category_unknown',
  # 'manufacturer_category_unknown',
  # 'is_fast'
]

In [0]:
significant_features = [feature for feature in significant_features if feature not in significcant_features_to_remove]

pprint.pprint(significant_features)
print("--"*50)
print("  "*50)
print("--"*50)
print(f"Number of significant features after removing collinear features:{len(significant_features)}")

In [0]:
# STEP 1: Calculate VIF for all features

print("="*80)
print("🔍 VIF MULTICOLLINEARITY ANALYSIS")
print("="*80)

X = user_info_df[significant_features].copy()
X_clean = X.dropna()

print(f"\nFeatures: {len(significant_features)}")
print(f"Observations: {len(X_clean)} (dropped {len(X) - len(X_clean)} with NaN)")

# Calculate VIF
vif_results = []
for i, col in enumerate(significant_features):
    try:
        vif_value = variance_inflation_factor(X_clean.values, i)
    except Exception as e:
        vif_value = np.inf
    vif_results.append({'feature': col, 'VIF': vif_value})

vif_df = pd.DataFrame(vif_results)

# Add flags
def get_flag(vif):
    if np.isinf(vif):
        return '🚨 INFINITE'
    elif vif > 10:
        return '⚠️ HIGH'
    elif vif > 5:
        return '⚡ MODERATE'
    else:
        return '✓ OK'

vif_df['flag'] = vif_df['VIF'].apply(get_flag)
vif_df = vif_df.sort_values('VIF', ascending=False)


In [0]:
# STEP 2: Summary and flagged features

n_infinite = np.isinf(vif_df['VIF']).sum()
n_high = ((vif_df['VIF'] > 10) & ~np.isinf(vif_df['VIF'])).sum()
n_moderate = ((vif_df['VIF'] > 5) & (vif_df['VIF'] <= 10)).sum()
n_ok = (vif_df['VIF'] <= 5).sum()

print(f"\n📊 VIF Summary:")
print(f"   🚨 Infinite (perfect multicollinearity): {n_infinite}")
print(f"   ⚠️  High (VIF > 10):                      {n_high}")
print(f"   ⚡ Moderate (VIF 5-10):                   {n_moderate}")
print(f"   ✓  OK (VIF ≤ 5):                          {n_ok}")

# Show problematic features
print("\n" + "-"*60)
print("FEATURES TO REVIEW:")
print("-"*60)

if n_infinite > 0:
    print("\n🚨 INFINITE VIF (must remove one from each correlated group):")
    display(vif_df[np.isinf(vif_df['VIF'])][['feature', 'VIF', 'flag']])

if n_high > 0:
    print("\n⚠️  HIGH VIF (consider removing):")
    display(vif_df[(vif_df['VIF'] > 10) & ~np.isinf(vif_df['VIF'])][['feature', 'VIF', 'flag']])

if n_moderate > 0:
    print("\n⚡ MODERATE VIF (monitor, usually OK to keep):")
    display(vif_df[(vif_df['VIF'] > 5) & (vif_df['VIF'] <= 10)][['feature', 'VIF', 'flag']])

#### Stage 2 Models

In [0]:
results = run_all_models(
    df=user_info_df,
    feature_cols=significant_features,
    outcome_var="wonky_study_count",
    user_id_var="respondentPk",
    include_logistic=True,
    include_vif=True,
    rf_n_estimators=100,
    lr_regularization_C=1.0,
)

In [0]:
results = run_shap_analysis(
    results=results,
    df=user_info_df,
    sample_size=1000,
)

#### Model Insights

In [0]:
print("\n" + "="*80)
print("📈 LINEAR REGRESSION RESULTS")
print("="*80)
print("Interpretation: Coefficient = change in wonky_study_count when feature=1 vs 0")
print("Stars: *** p<0.001, ** p<0.01, * p<0.05\n")

linear_df = results['linear_importance'].copy()
linear_df['sig'] = linear_df['p_value'].apply(
    lambda x: '***' if x < 0.001 else ('**' if x < 0.01 else ('*' if x < 0.05 else ''))
)
linear_df['coef_display'] = linear_df.apply(
    lambda row: f"{row['coefficient']:+.4f}{row['sig']}", axis=1
)

display(linear_df.head(20)[['feature', 'coef_display', 'p_value', 'importance']])


In [0]:
if 'lr_importance' in results:
    print("\n" + "="*80)
    print("🎯 LOGISTIC REGRESSION - ODDS RATIOS")
    print("="*80)
    print("""
Odds Ratio Interpretation:
  • OR = 1.0:  No effect
  • OR = 1.5:  50% more likely to be wonky when feature=1
  • OR = 2.0:  2x more likely (100% increase)
  • OR = 0.5:  50% less likely (half the odds)
""")
    
    lr_df = results['lr_importance'].copy()
    
    # Add interpretation
    def interpret_or(x):
        if pd.isna(x) or x <= 0 or x > 100 or x < 0.01:
            return "⚠️ Extreme"
        if abs(x - 1) < 0.05:
            return "No effect"
        if x > 1:
            pct = (x - 1) * 100
            return f"↑ {pct:.0f}% more likely" if pct <= 100 else f"↑ {x:.1f}x"
        else:
            return f"↓ {(1-x)*100:.0f}% less likely"
    
    lr_df['interpretation'] = lr_df['lr_odds_ratio'].apply(interpret_or)
    
    # Filter valid odds ratios
    valid_lr = lr_df[(lr_df['lr_odds_ratio'] >= 0.01) & (lr_df['lr_odds_ratio'] <= 100)]
    
    display(valid_lr.head(20)[['feature', 'lr_coefficient', 'lr_odds_ratio', 
                               'interpretation', 'lr_p_value']])

In [0]:
print("\n" + "="*80)
print("🌲 RANDOM FOREST IMPORTANCE")
print("="*80)

rf_df = results['rf_importance'].copy()
display(rf_df.head(20)[['feature', 'rf_importance', 'rf_importance_pct']])

In [0]:
if 'shap_importance' in results:
    print("\n" + "="*80)
    print("🔮 SHAP FEATURE IMPORTANCE")
    print("="*80)
    print("""
SHAP shows HOW features affect predictions:
  • shap_importance: How much the feature matters
  • shap_mean: Direction (+ increases wonky, - decreases)
  • shap_direction: Plain English interpretation
""")
    
    shap_df = results['shap_importance'].copy()
    display(shap_df.head(20)[['feature', 'shap_importance', 'shap_mean', 'shap_direction']])

In [0]:
# Beeswarm plot - shows direction and distribution
plot_shap_summary(results, max_display=20)

In [0]:
plot_shap_bar(results, max_display=20)

In [0]:
# # to explore specific features using shap

# # How does is_saturday affect predictions?
# plot_shap_dependence(results, 'is_saturday')

# # With interaction coloring
# plot_shap_dependence(results, 'is_saturday', interaction_feature='is_early_morning')

In [0]:
print("\n" + "="*80)
print("🔍 MULTICOLLINEARITY CHECK (VIF)")
print("="*80)
print("""
VIF Interpretation:
  • VIF < 5:  ✓ No concern
  • VIF 5-10: ⚡ Moderate
  • VIF > 10: ⚠️ High - consider removing
""")

vif_df = results['vif_data'].copy()
vif_df['status'] = vif_df['VIF'].apply(
    lambda x: '✓ OK' if x < 5 else ('⚡ Moderate' if x < 10 else '⚠️ HIGH')
)

high_vif = vif_df[vif_df['VIF'] >= 5]
if len(high_vif) > 0:
    print(f"Features with VIF >= 5:")
    display(high_vif[['feature', 'VIF', 'status']])
else:
    print("✓ All features have VIF < 5")

In [0]:
print("\n" + "="*80)
print("📋 STAKEHOLDER REPORT")
print("="*80)

stakeholder_report = create_stakeholder_report(results)
display(stakeholder_report.head(20))

In [0]:
print("\n" + "="*80)
print("💡 KEY INSIGHTS")
print("="*80)

# Top consensus features
print("\n🏆 TOP 5 CONSENSUS FEATURES:")
for i, (_, row) in enumerate(results['comparison'].head(5).iterrows(), 1):
    print(f"\n{i}. {row['feature']}")
    print(f"   Linear coef: {row['linear_coef']:+.4f} (p={row['linear_p_value']:.4e})")
    print(f"   RF importance: {row['rf_importance_pct']:.2f}%")
    if 'lr_odds_ratio' in row and pd.notna(row['lr_odds_ratio']):
        print(f"   Odds Ratio: {row['lr_odds_ratio']:.3f}")
    if 'shap_direction' in row:
        print(f"   SHAP: {row['shap_direction']}")

# Features that increase vs decrease wonky
if 'shap_importance' in results:
    shap_df = results['shap_importance']
    
    increases = shap_df[shap_df['shap_mean'] > 0.001].head(5)
    decreases = shap_df[shap_df['shap_mean'] < -0.001].head(5)
    
    print("\n🔴 FEATURES THAT INCREASE WONKY:")
    for _, row in increases.iterrows():
        print(f"   {row['feature']}: +{row['shap_mean']:.4f}")
    
    print("\n🔵 FEATURES THAT DECREASE WONKY:")
    for _, row in decreases.iterrows():
        print(f"   {row['feature']}: {row['shap_mean']:.4f}")

In [0]:
print("\n" + "="*80)
print("🎯 HIGH-CONFIDENCE FEATURES")
print("="*80)

high_confidence = get_significant_features(results, p_threshold=0.05, min_rf_importance=0.01)
print(f"\nFeatures significant in Linear + top RF importance: {len(high_confidence)}")
for f in sorted(high_confidence):
    print(f"  ✓ {f}")

In [0]:
print("\n" + "="*80)
print("💾 EXPORT")
print("="*80)

# Export to CSV
export_df = export_results(results, 'feature_importance_results.csv')

# Also save as parquet for downstream use
results['comparison'].to_parquet('modelling_comparison.parquet', index=False)
print("✓ Results saved to modelling_comparison.parquet")

In [0]:
# Interactive Plotly chart comparing Linear vs RF importance

comparison_df = results['comparison'].head(20).copy()

fig = go.Figure()

# Linear coefficients
fig.add_trace(go.Bar(
    x=comparison_df['linear_coef'],
    y=comparison_df['feature'],
    orientation='h',
    name='Linear Coefficient',
    marker=dict(color='#1f77b4'),
))

# RF importance (scaled for visibility)
rf_scale = comparison_df['linear_coef'].abs().max() / comparison_df['rf_importance'].max()
fig.add_trace(go.Bar(
    x=comparison_df['rf_importance'] * rf_scale,
    y=comparison_df['feature'],
    orientation='h',
    name=f'RF Importance (scaled)',
    marker=dict(color='#ff7f0e'),
))

fig.update_layout(
    title="Top 20 Features: Linear Coefficient vs RF Importance",
    xaxis_title='Value',
    yaxis_title='Feature',
    barmode='group',
    yaxis=dict(autorange='reversed'),
    height=600,
    margin=dict(l=200),
)

fig.show()

In [0]:
# The 'results' dictionary contains everything:
# - results['linear_model']: OLS model
# - results['rf_model']: Random Forest model  
# - results['lr_model']: Logistic Regression model
# - results['comparison']: Unified comparison table
# - results['shap_values']: SHAP values array
# - results['shap_explainer']: SHAP explainer object